In [213]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##### TODO #########################################
### RENAME THIS FILE TO YOUR TEAM NAME #############
### IMPLEMENT 'getMyPosition' FUNCTION #############
### TO RUN, RUN 'eval.py' ##########################

nInst = 50
currentPos = numpy.zeros(nInst)

In [214]:
# Simple Moving Average
# We can do this with convolutions from COMP3821 haha
def sma(data, window):
    # weights array is an array of '1/window'. Multiplying it with the data array will give us the 
    # moving average since '1/window * data' describes the average of each datapoint
    weights = np.repeat(1.0, window)/window
    smas = np.convolve(data, weights, 'valid')
    return smas

In [215]:
def getMyPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape
    if (nt < 2):
        return np.zeros(nins)
    lastRet = np.log(prcSoFar[:, -1] / prcSoFar[:, -2])
    lNorm = np.sqrt(lastRet.dot(lastRet))
    lastRet /= lNorm
    rpos = np.array([int(x) for x in 5000 * lastRet / prcSoFar[:, -1]])
    currentPos = np.array([int(x) for x in currentPos+rpos])
    return currentPos

For some reason, this Notebook code block is outputting a different score each time it is run?

What???

For now, develop the algorithm in the python file itself. Just use the Notebook for analysis and testing hypothesis.

In [216]:
nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000


def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T


pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))


def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(250, 501):
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)


(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)


Loaded 50 instruments for 500 days
Day 250 value: -498.92 todayPL: $-498.92 $-traded: 498924 return: -0.00100
Day 251 value: -771.30 todayPL: $-272.37 $-traded: 501256 return: -0.00154
Day 252 value: -1248.26 todayPL: $-476.96 $-traded: 503615 return: -0.00248
Day 253 value: -1393.86 todayPL: $-145.61 $-traded: 505514 return: -0.00276
Day 254 value: -1880.65 todayPL: $-486.79 $-traded: 507323 return: -0.00371
Day 255 value: -1882.54 todayPL: $-1.89 $-traded: 508929 return: -0.00370
Day 256 value: -1662.40 todayPL: $220.13 $-traded: 510914 return: -0.00325
Day 257 value: -1599.91 todayPL: $62.49 $-traded: 512833 return: -0.00312
Day 258 value: -1870.47 todayPL: $-270.56 $-traded: 514874 return: -0.00363
Day 259 value: -1871.43 todayPL: $-0.95 $-traded: 516856 return: -0.00362
Day 260 value: -1649.75 todayPL: $221.67 $-traded: 519625 return: -0.00317
Day 261 value: -942.36 todayPL: $707.40 $-traded: 521299 return: -0.00181
Day 262 value: -773.17 todayPL: $169.19 $-traded: 523502 return: 